In [1]:
import random

In [2]:
random.randint(1, 5)

1

In [10]:
def simulate(n, m, s=1000):
    values = []
    for i in range(s):
        dice = []
        for j in range(n):
            roll = random.randint(1,m)
            dice.append(roll)
        values.append(sum(dice))
    return values

In [11]:
import hvplot.pandas
import pandas as pd
import panel as pn
import holoviews as hv

In [12]:
df = pd.DataFrame(simulate(2, 6))

In [13]:
pn.Row(df.hvplot.kde() * hv.VLine(df.mean()[0]), pn.Column("Mean: ", df.mean()[0]))

Row
    [0] HoloViews(Overlay)
    [1] Column
        [0] Markdown(str)
        [1] Str(float64)

In [15]:
import param as pm
import hvplot.pandas
import pandas as pd
import panel as pn
import holoviews as hv
import random

class DiceRolls(pm.Parameterized):
    n = pm.Number(2, bounds=(1,20), step=1)
    m = pm.Number(6, bounds=(1,20), step=1)
    
    def simulate(self, s=1000):
        n = self.n
        m = self.m
        values = []
        for i in range(s):
            dice = []
            for j in range(n):
                roll = random.randint(1,m)
                dice.append(roll)
            values.append(sum(dice))
        return values
    
    def view(self):
        df = pd.DataFrame(self.simulate())
        return pn.Row(df.hvplot.kde() * hv.VLine(df.mean()[0]), pn.Column("Mean: ", df.mean()[0]))

dr = DiceRolls()
pn.Row(dr, dr.view)

Row
    [0] Column(margin=5, name='DiceRolls', width=300)
        [0] StaticText(value='<b>DiceRolls</b>')
        [1] FloatSlider(end=20, name='N', start=1, step=1, value=2)
        [2] FloatSlider(end=20, name='M', start=1, step=1, value=6)
    [1] ParamMethod(method)

In [13]:
df.mean()

0    9.077
dtype: float64

In [38]:
results = []
for n in range(2,40):
    for m in range(2,40):
        sim_vals = simulate(n,m)
        mean = sum(sim_vals) / len(sim_vals)
        results.append([n,m,mean])

In [41]:
results = pd.DataFrame(results, columns=["n", "m", "Mean"])

In [42]:
results

,n,m,Mean
0,2,2,3.032
1,2,3,4.022
2,2,4,4.982
3,2,5,6.003
4,2,6,6.966
...,...,...,...
1439,39,35,702.764
1440,39,36,721.237
1441,39,37,739.181
1442,39,38,761.794


In [ ]:
@np.vectorize
def simulate(n, m, s=1000):
    values = []
    for i in range(s):
        dice = []
        for j in range(n):
            roll = random.randint(1,m)
            dice.append(roll)
        values.append(sum(dice))
    return sum(values) / len(values)

In [54]:
n_range = range(2,10)

In [55]:
m_range = range(2,10)

In [56]:
import numpy as np

In [57]:
phase_space = np.meshgrid(n_range, m_range)

In [58]:
phase_space[0]

array([[2, 3, 4, 5, 6, 7, 8, 9],
       [2, 3, 4, 5, 6, 7, 8, 9],
       [2, 3, 4, 5, 6, 7, 8, 9],
       [2, 3, 4, 5, 6, 7, 8, 9],
       [2, 3, 4, 5, 6, 7, 8, 9],
       [2, 3, 4, 5, 6, 7, 8, 9],
       [2, 3, 4, 5, 6, 7, 8, 9],
       [2, 3, 4, 5, 6, 7, 8, 9]])

In [59]:
phase_space[1]

array([[2, 2, 2, 2, 2, 2, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 3],
       [4, 4, 4, 4, 4, 4, 4, 4],
       [5, 5, 5, 5, 5, 5, 5, 5],
       [6, 6, 6, 6, 6, 6, 6, 6],
       [7, 7, 7, 7, 7, 7, 7, 7],
       [8, 8, 8, 8, 8, 8, 8, 8],
       [9, 9, 9, 9, 9, 9, 9, 9]])

In [61]:
value = simulate(*phase_space)

In [ ]:
        fee_range = np.linspace(*s.param['seller_fee_percentage'].bounds, num=120)
        cost_range = np.linspace(*s.param['average_cost_of_returns'].bounds, num=120)
        phase_space = np.meshgrid(fee_range, cost_range)
        collected_fees = self.get_collected_fees(phase_space[0])
        return_costs = self.get_return_costs(phase_space[1])
        value_delta = collected_fees - return_costs
        return fee_range, cost_range, value_delta 

In [63]:
df = hv.Dataset((n_range, m_range, value), ["n", "m"], "mean").dframe()

In [66]:
hv.HeatMap(df)

:HeatMap   [n,m]   (mean)

In [64]:
hv.Image(df).opts(
            title="Cost of Returns and Seller Fee ODE",
            height=500, width=650,
            colorbar=True, clabel='Delta Network Vale Per Timestep', cmap=self.colormap, color_levels=color_levels,
            tools=['hover']
        )

DataError: Image type expects gridded data, PandasInterface is columnar. To display columnar data as gridded use the HeatMap element or aggregate the data (e.g. using rasterize or np.histogram2d).

In [ ]:
    def get_view_components(self):
        fee_range, cost_range, value_delta  = s.get_value_delta_phase()
        dataset = hv.Dataset((fee_range, cost_range, value_delta), ['Protocol Fee', 'Cost of a return'], 'Delta Network Rewards')
        df = dataset.dframe()
        boundary = df[abs(df['Delta Network Rewards']) < np.diff(value_delta).mean()]
        color_bound = max(abs(df['Delta Network Rewards']))
        return dataset, boundary, color_bound
    
    def view_value_delta_phase(self):
        dataset, boundary, color_bound = self.get_view_components()
        color_levels = list(np.linspace(-color_bound, color_bound, 120))
        value_delta_phase_view = hv.Image(dataset).opts(
            title="Cost of Returns and Seller Fee ODE",
            height=500, width=650,
            colorbar=True, clabel='Delta Network Vale Per Timestep', cmap=self.colormap, color_levels=color_levels,
            tools=['hover']
        )
        boundary_view = boundary.hvplot.line(x='Protocol Fee', y='Cost of a return', color='white', line_width=16, alpha=0.5)
        point_view = pd.DataFrame([s.seller_fee_percentage, s.average_cost_of_returns]).T.hvplot.scatter(x='0', y='1')
        return value_delta_phase_view * boundary_view * point_view